In [11]:
# get all noodle headlines from their wordpress site

import requests
from bs4 import BeautifulSoup
import pandas as pd
import csv

url = "https://thelocalnoodle.com/page/"


def get_headlines(page):
    r = requests.get(page)
    return parse_headlines(r.content)

def parse_headlines(html_doc):
    soup = BeautifulSoup( html_doc , 'html.parser') 
    headlines = []
    # for each entry that I find
    for e in soup.find_all("h1", {"class": "entry-title"}):
        # get the text and put it into a list
        headlines.append(e.text)
            
    return headlines
    

    
noodle_headlines = []
for i in range(1,15):
    url_i = url + str(i) + "/"
    noodle_headlines += get_headlines(url_i) 
    

In [12]:
# now let's use the campus sitemap to get all of its headlines. the noodle started
# in september of 2016, so let's start our scrape there.

begin = 35088
url = 'https://middleburycampus.com/'
# the two sitemaps which include articles from our desired date range
categories = ['sports','news','local','opinion','arts-academics']

def get_articles(maps):
    articles = []
    
    for m in maps:
        r = requests.get(url + m)
        soup = BeautifulSoup( r.content , 'html.parser')
        for e in soup.find_all("loc"):
            # get the text and put it into a list
            
            articles.append(e.text[29:]) # exclude the articl
            
    return parse_articles(articles)

    
def parse_articles(articles):
    # use the URL to extract the title of the article
    headlines = []
    
    for a in articles:
        t = a.split('/')
        if ( int(t[0]) > begin) and (t[1] in categories):
            headline = t[2].replace('-',' ')
            headlines.append(headline)
            
    return headlines
    
maps = ['wp-sitemap-posts-post-7.xml','wp-sitemap-posts-post-8.xml']
campus_headlines = get_articles(maps)


In [19]:
# save it as a dataframe
df1 = pd.DataFrame({'text':noodle_headlines})
df1['label'] = 1
df2 = pd.DataFrame({'text':campus_headlines})
df2['label'] = 0


# combine and shuffle 
df = pd.concat([df1,df2])
df = df.sample(frac=1).reset_index(drop=True)

# Converting all text to lowercase, fixing ampersands and getting rid
# of dashes and apostrophes as they can mess up the dictionary
df['text'] = df['text'].str.lower()
df['text'] = df['text'].str.replace(r'&amp;', 'and')
df['text'] = df['text'].str.replace(r'-', ' ')
df['text'] = df['text'].str.replace(r'[^\s\w]','')
df.head()

,text,label
0,weekend wins for womens soccer,0
1,town seeks public input on green spaces ahead ...,0
2,untitled romantic comedy,0
3,he was too good remembering former college emp...,0
4,re coronavirus in n y manhattan woman is first...,0


In [20]:
df.to_csv('CampusOrNoodle.csv')